# Election 2016
___
2018 | Bernard Kung
___

A fun exploration of 2016 US election Voting-Age Population (VAP) changes compared against 2012. Let's also build...choropleths!

### Initializing Workspace
___

In [97]:
import pandas as pd
import numpy as np
import plotly.plotly as py
import plotly.graph_objs as go 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [98]:
init_notebook_mode(connected=True) 

### Loading and Cleaning Data
___

Data for this project is from the United States Elections Project [[1](#Sources)]. 

I'm going to build two dataframes, **elect2016_data** and **elect2012_data**, for each respective year. I also do a little cleaning: 

* For 2016 election data I need to coerce some columns; I go into more detail in the **election2016_turnout** notebook.
* For 2012 election data a little less work is needed due to different formatting. 

On a more involved project I might do a little more work syncing columns and filling NaNs; I'm just trying to make some cool plots to satisfy my curiosity.

In [99]:
elect2016_data = pd.read_csv(r'..\data\2016_November_General_Election.csv',
                            header= 1, nrows= 52, thousands=r',')
elect2016_data.rename(columns={'Unnamed: 0':'State'}, inplace= True)
elect2016_data.rename(columns=lambda x: x.replace(' ',''), inplace= True)
percent_data = elect2016_data[['VEPTotalBallotsCounted', 'VEPHighestOffice', 'VAPHighestOffice', '%Non-citizen']]
for cols in  percent_data.columns:
    percent_data.loc[percent_data[cols].notnull(),cols] = percent_data.loc[percent_data[cols].notnull(), cols].apply(lambda x: x.replace('%',''))
percent_data = percent_data.astype(np.float64)/100
elect2016_data[percent_data.columns] = percent_data

E:\Anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [100]:
elect2012_data = pd.read_csv(r'..\data\2012_November_General_Election.csv',
                            header= 1, nrows= 52, thousands=r',')
elect2012_data.rename(columns=lambda x: x.replace(' ',''), inplace= True)

In [101]:
elect2016_data.head()

,State,StateResultsWebsite,Status,VEPTotalBallotsCounted,VEPHighestOffice,VAPHighestOffice,TotalBallotsCounted(Estimate),HighestOffice,Voting-EligiblePopulation(VEP),Voting-AgePopulation(VAP),%Non-citizen,Prison,Probation,Parole,TotalIneligibleFelon,OverseasEligible,StateAbv
0,United States,NaN,NaN,0.602,0.593,0.547,138846571.0,136700729,230585915,250055734,0.084,1456032,2254727,508576,3249802,4739596.0,NaN
1,Alabama,http://www.alabamavotes.gov/downloads/election...,Official,0.593,0.590,0.563,2134061.0,2123372,3601361,3770142,0.026,30627,56700,8138,71084,NaN,AL
2,Alaska,http://www.elections.alaska.gov/results/16GENR/,Official,0.618,0.613,0.574,321271.0,318608,519849,555367,0.043,5338,7077,2210,11582,NaN,AK
3,Arizona,http://apps.azsos.gov/election/2016/General/Of...,Official,0.562,0.550,0.489,2661497.0,2604657,4734313,5331034,0.095,38068,76005,7379,88770,NaN,AZ
4,Arkansas,http://results.enr.clarityelections.com/AR/639...,Official,0.531,0.528,0.494,1137772.0,1130635,2142571,2286625,0.038,17405,28900,23093,56971,NaN,AR


In [102]:
elect2012_data.head()

,State,VEPTotalBallotsCounted,VEPHighestOffice,VAPHighestOffice,TotalBallotsCounted,HighestOffice,Voting-EligiblePopulation(VEP),Voting-AgePopulation(VAP),%Non-citizen,Prison,Probation,Parole,TotalIneligibleFelon,OverseasEligible,StateAbv
0,United States,0.586,0.580,0.536,130292355.0,129070906,222474111,240957993,0.084,1473412,2342582,637410,3352390,5127418.0,NaN
1,Alabama,NaN,0.586,0.560,NaN,2074338,3539217,3707440,0.026,32232,57993,8616,71584,NaN,AL
2,Alaska,0.589,0.587,0.553,301694.0,300495,511792,543763,0.038,5633,7173,1882,11317,NaN,AK
3,Arizona,0.530,0.526,0.465,2323579.0,2306559,4387900,4959270,0.099,35188,72452,7460,81048,NaN,AZ
4,Arkansas,0.511,0.507,0.477,1078548.0,1069468,2109847,2242740,0.035,14471,30122,23372,53808,NaN,AR


### Turnout Changes
___

The original question I was interested in answering is how has voter turnout changed between 2012 and 2016. I'll focus on:

* **VAPHighestOffice**: This is the change in ratio of Highest Office to Voting-Age Population (VAP) from 2012 to 2016. This shows the change in turnout rates, which will normalize against changes in population. 
* **VAP**: This is the change in Voting-Age Population from 2012 to 2016, just for reference.

Together, these two graphs should complement each other and provide a decent picture.

In [103]:
change_data = pd.DataFrame(
    data = {'State' : elect2016_data['State'],
            'StateAbv' : elect2016_data['StateAbv'],
            'VAP' : elect2016_data['Voting-AgePopulation(VAP)']/elect2012_data['Voting-AgePopulation(VAP)'],
            'VAPHighestOffice' : elect2016_data['VAPHighestOffice']/elect2012_data['VAPHighestOffice']})

In [105]:
VAP_data = dict(type = 'choropleth',
            colorscale = 'Bluered',
            reversescale = False,
            locations = change_data['StateAbv'],
            locationmode= 'USA-states',
            text= change_data['State'],
            z= change_data['VAPHighestOffice'],
            colorbar = {'title': '% Turnout Rate'})
VAP_layout = dict (geo= {'scope':'usa'}, title= 'Change in Voter Turnout Rates from (2012-2016)')
VAP_choromap = go.Figure(data = [VAP_data],layout = VAP_layout)

In [112]:
iplot(VAP_choromap)

In [107]:
percent_data = dict(type = 'choropleth',
            colorscale = 'Bluered',
            reversescale = False,
            locations = change_data['StateAbv'],
            locationmode= 'USA-states',
            text= change_data['State'],
            z= change_data['VAP'],
            colorbar = {'title': '% VAP'})
percent_layout = dict (geo= {'scope':'usa'}, title= 'Change in Voting-Age Population (VAP) (2016-2012)')
percent_choromap = go.Figure(data = [percent_data],layout = percent_layout)

In [113]:
iplot(percent_choromap)

Looking between the two graphs, I'm actually most interested in how they correspond to each other. Considering the color in the top graph to the color in the bottom graph:

* Red-Red States: These states saw increases in both turnout rates and population. This can be interpreted as an overall more involved population.
* Red-Blue States: These states saw an increase in turnout rate despite slower growth or reduction in population. This can be interpreted as a state waning in relative population but increasing political involvement. 
* Blue-Red States: These states saw a decrease in turnout rate despite higher growth in population. This can be interpreted as an indication of withdrawal from national politics.
* Blue-Blue States: These states saw decreases in both turnout rates and population. This can be interpreted as a state generally waning overall.

Obviously these are very off-the-cuff interpretations to be based on solely the presidential vote. Likely including more data on US Senators and Representatives, as well as state politicians, would help guide this interpretation. 

In [109]:
change_data['Comparison'] = change_data['VAPHighestOffice']/change_data['VAP']

In [110]:
cross_data = dict(type = 'choropleth',
            colorscale = 'Viridis',
            reversescale = False,
            locations = change_data['StateAbv'],
            locationmode= 'USA-states',
            text= change_data['State'],
            z= change_data['Comparison'],
            colorbar = {'title': '% Rate/VAP'})
cross_layout = dict (geo= {'scope':'usa'}, title= 'Change in Turnout Rate to Voting-Age Population (VAP) Ratio (2016-2012)')
cross_choromap = go.Figure(data = [cross_data],layout = cross_layout)

In [114]:
iplot(cross_choromap)

This third graph plots the ratio of turnout rate change to VAP change.

* Yellow states saw increasing turnout rates, decreasing populations.
* Purple states saw decreasing turnout rates, increasing populations.
* Teal states would be those whose turnout rates and populations changed at the same rate, either increasing or decreasing together.

Some fairly loose interpretation. The logical next step would be to start looking at state-by-state Republican-Democrat voter ratios.

### Sources 
___
McDonald, Michael P. "2016 November General Election Turnout Rates" United States Elections Project. http://www.electproject.org/2016g



___
2018 | Bernard Kung
___